In [3]:
#######################################################
#### Set this value to the directory of period-suite ##
#######################################################
pathToSuite="/Users/sertoez/git-projects/period-suite/"
#######################################################

import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
defaultPrecision=1e-30;
# Do not forget to increase the precision of initial conditions if going beyond e-100
#
#
#
precision=[]; #if empty we will use the default precision for all systems 
paths=[]; #unless overridden we will convert this to a list of straight paths

t0=time.time();
load(pathToSuite+"current.sage")
t1=time.time();
print "Loaded file in", t1-t0, "seconds";
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "\nIntegrating system number", j;
    transitionMatricies = []
    for i in [1..noOfdeqs]:
        tt0=time.time()
        tm=deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true)
        print "\tODE number", i, "took", time.time()-tt0, "seconds to integrate"
        transitionMatricies=transitionMatricies+[tm]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    print "The largest error:"
    print max([max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies])
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(2000), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:noOfdeqs]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print Matrix(periods).str()



# Write the periods to file

def row2string(row):
    strRow=str(row[0])
    for i in [1..(len(row)-1)]:
        strRow=strRow+", "+str(row[i])
    return "["+strRow+"]"
    
outputFile = open(pathToSuite+"lastPeriods",'w')

# Change default precision here or in Magma if you want to load them
outputFile.write("SetDefaultRealFieldPrecision(100);\n")
outputFile.write("CC<I>:=ComplexField(100);\n")
outputFile.write("M:=Matrix(["+row2string(periods[0]))
for i in [1..(periods.nrows()-1)]:
    outputFile.write(",\n"+row2string(periods[i]))
outputFile.write("]);")
outputFile.close()


Loaded file in 0.0337181091309 seconds
Integrating systems 1 through 1

Integrating system number 1
	ODE number 1 took 0.377887964249 seconds to integrate
	ODE number 2 took 0.420356988907 seconds to integrate
	ODE number 3 took 0.647004127502 seconds to integrate
Maximal error in transition matricies:
[6.5435555e-31, 6.5435555e-31, 8.9306353e-29]
The largest error:
8.9306353e-29

Accumulated maximal error: 1.2500746e-29

It took 1.48295307159 seconds in total.

[    1.781150396113309967409093856902 + 1.259463523404826719290380559488*I  -0.8905751980566549837045469284510 + 0.8905751980566549837045469284510*I  -0.8905751980566549837045469284510 - 0.8905751980566549837045469284510*I                                         -1.259463523404826719290380559488                                       -1.259463523404826719290380559488*I                                          1.259463523404826719290380559488]
[    1.781150396113309967409093856902 + 1.259463523404826719290380559488*I  -0.89057519

In [8]:
#### Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 80
##
## verify that you found honest relations by checking suspected relations using more digits
##
verify= 90
##
##

load(pathToSuite+"IntersectionMatrix.sage")

periodVector=periods
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])

proj = M.transpose()
Proj = (10^scale*proj).apply_map(lambda x : 10^3*x.round())
lattice = block_matrix([[Proj, matrix.identity(Proj.dimensions()[0])]])
reducedLattice = lattice.LLL()

Proj2 = (10^verify*proj).apply_map(lambda x : x.round())
testRelations = [v[2:23] * Proj2 for v in reducedLattice.rows()]

norms1 = [reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]
norms2 = [testRelations[i].norm().n()+reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]

norms=norms2

sortedNorms=sorted(norms)
consecutiveRatios = [sortedNorms[i]/sortedNorms[i-1] for i in [1..20]]

j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if sortedNorms[0] > 50 :
    rank = 1

print "\nPicard rank appears to be", rank, "\n"
print "Check to see if this makes sense:\n"
print "First row records the norm of the relation."
print "Second row shows if the relation continues to hold with more digits.\n"

print "01","||", "---hyperplane section---"
j=0
while j <= 7 :
    print "0" + str(j+2), "||", norms1[j], testRelations[j]
    j=j+1
while j <= 20 :
    print j+2, "||", norms1[j], testRelations[j]
    j=j+1



Picard rank appears to be 4 

Check to see if this makes sense:

First row records the norm of the relation.
Second row shows if the relation continues to hold with more digits.

01 || ---hyperplane section---
02 || 12.0000000000000 (0, -1)
03 || 4.24264068711929 (-1, 0)
04 || 3.16227766016838 (1, 0)
05 || 8.84714407807121e8 (434019835419289714, -1579987910859333822)
06 || 9.41387220089127e8 (1278906723460583439, 2449580178737802349)
07 || 1.04197265339657e9 (-1282132221586811880, 685069429064135325)
08 || 1.11731943805812e9 (2716224112764909592, -5532881672586495933)
09 || 1.06469020984576e9 (-2081888593794467603, -5748607049712836136)
10 || 9.94685675984435e8 (-3194871860928184132, -104377323627059228)
11 || 1.09433933986478e9 (274827116, 4899160357407852265)
12 || 1.11221158688794e9 (2282204278131302813, 3952893761908041313)
13 || 9.08579715316538e8 (-3762276975234839341, 194234405440588099)
14 || 1.11833018156844e9 (-232853861299916673, -4794024327980815344)
15 || 1.02454223855214

In [11]:
reducedLattice.row(17)[2:23]

(1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0)